In [1]:
import os
import sys
sys.path.append('../..')

import pytorch_lightning as pl

from lightning_modules.image_segmentation import ImageSegmentationModule
from utils.io import load_yaml


# prefix = '/home/martin/Tresors/homography_imitation_learning_logs/boundary_image_segmentation/unet/resnet/34/tiny/version_4'
# checkpoint = 'epoch=192-step=771.ckpt'
# prefix = '/home/martin/Tresors/homography_imitation_learning_logs/boundary_image_segmentation/unet/efficient_net/b0/version_0'
# checkpoint = 'epoch=28-step=869.ckpt'
# prefix = '/home/martin/Tresors/homography_imitation_learning_logs/boundary_image_segmentation/deeplabv3p/efficient_net/b0/version_0'
# checkpoint = 'epoch=49-step=1499.ckpt'
prefix = '/home/martin/Tresors/homography_imitation_learning_logs/boundary_image_segmentation/unet/resnet/34/version_0'
checkpoint = 'epoch=35-step=1079.ckpt'
# prefix = '/home/martin/Tresors/homography_imitation_learning_logs/boundary_image_segmentation/deeplabv3p/resnet/34/version_0'
# checkpoint = 'epoch=45-step=1379.ckpt'

config = load_yaml(os.path.join(prefix, 'config.yml'))

model = ImageSegmentationModule(**config['model'])
model = model.load_from_checkpoint(os.path.join(prefix, 'checkpoints', checkpoint), **config['model'])
model = model.eval()
model.freeze()


/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/ipykernel/pylab/backend_inline.py:164: DeprecationWarning: `configure_inline_support` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.configure_inline_support()`
  configure_inline_support(ip, backend)
/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/matplotlib_inline/config.py:66: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [2]:
import cv2
from kornia.geometry import resize
from kornia import image_to_tensor, tensor_to_image
import torch

from utils.io import recursive_scan2df
from utils.sampling import ConsecutiveSequences, RandomSequences
from utils.processing import endoscopy

prefix = '/media/martin/Samsung_T5/data/endoscopic_data/cholec80_splits'
# prefix = '/media/martin/Samsung_T5/data/endoscopic_data/ROBUST_MIS_raw_data'
df = recursive_scan2df(prefix, '.mp4')
# df = recursive_scan2df(prefix, '.avi')
N = 10

if torch.cuda.is_available():
    model = model.to('cuda')

paths = [os.path.join(prefix, row.folder, row.file) for _, row in df.sample(N).iterrows()]

cs = RandomSequences(max_seq=100, paths=paths, seq_len=100)
for frames in cs:
    frames = image_to_tensor(frames)
    frames = frames.to(model.device)
    frames = frames.float()/255.
    # frames = resize(frames, [270, 480])
    segs = model(frames)
    avg_seg = segs.mean(axis=0)

    for seg, frame in zip(segs, frames):
        seg = tensor_to_image(avg_seg, keepdim=False)
        frame = tensor_to_image(frame, keepdim=False)

        center, radius = endoscopy.ransacBoundaryCircle(seg*255, th=10, fit='numeric', n_pts=100, n_iter=1)

        if radius is not None:
            center, radius = center.astype(int), int(radius)
        
            top_left, shape = endoscopy.maxRectangleInCircle(seg.shape, center, radius)
            top_left, shape = top_left.astype(int), tuple(map(int, shape))
            cv2.circle(frame, (center[1], center[0]), radius, color=(1, 1, 0), thickness=2)
            cv2.rectangle(frame, (top_left[1], top_left[0]), (top_left[1]+shape[1], top_left[0]+shape[0]), (1, 1, 0), thickness=2)

            # crop = endoscopy.crop(frame, top_left, shape)
            # cv2.imshow('crop', crop)

        mask = endoscopy.bilateralSegmentation(frame*255, th=0.01)
        center, radius = endoscopy.ransacBoundaryCircle(mask, th=10, fit='numeric', n_pts=100, n_iter=1)

        if radius is not None:
            center, radius = center.astype(int), int(radius)
            cv2.circle(frame, (center[1], center[0]), radius, color=(1, 0, 1), thickness=2)

        cv2.imshow('seg', seg)
        cv2.imshow('frames', frame)
        cv2.waitKey()



/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [6]:
import cv2
from kornia import image_to_tensor, tensor_to_image, resize
import time
import torch

from utils.io import recursive_scan2df
from utils.sampling import ConsecutiveSequences, RandomSequences
from utils.processing import endoscopy

prefix = '/media/martin/Samsung_T5/data/endoscopic_data/cholec80_splits'
# prefix = '/media/martin/Samsung_T5/data/endoscopic_data/ROBUST_MIS_raw_data'
df = recursive_scan2df(prefix, '.mp4')
# df = recursive_scan2df(prefix, '.avi')
N = 10
inferences = 200
frame_cnt = 0
total_time = 0

paths = [os.path.join(prefix, row.folder, row.file) for _, row in df.sample(N).iterrows()]
exit = False

if torch.cuda.is_available():
    model = model.to('cuda')
# model = model.to('cpu')

cs = RandomSequences(max_seq=100, paths=paths, seq_len=100)
for frames in cs:
    for frame in frames:
        if frame_cnt >= inferences - 1:
            exit = True
            break
        start = time.time()
        frame = image_to_tensor(frame, keepdim=False)
        frame = frame.to(model.device)
        frame = frame.float()/255.
        # frames = resize(frames, [270, 480])
        segs = model(frame)
        total_time += time.time() - start

        frame_cnt += 1
        print('\r{}, device: {}, hxw: {}x{}'.format(frame_cnt, model.device, frame.shape[-2], frame.shape[-1]), end='')
    if exit:
        break

print('\nTotal time: ', total_time)
print('Avg time: ', total_time/frame_cnt)




/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


199, device: cuda:0, hxw: 480x854
Total time:  1.2827739715576172
Avg time:  0.006446100359586016


In [ ]:
# UNet
# unet efficient_net b0
# avg CPU time: 0.059 ms
# avg GPU time: 0.008 ms
# deeplabv3plus efficient_net b0
# avg CPU time: 0.061 ms
# avg GPU time: 0.008 ms

# ResNet
# unet resnet 34
# avg CPU time: 0.083 ms
# avg GPU time: 0.004 ms
# deeplabv3plus resnet 34
# avg CPU time: 0.076 ms
# avg GPU time: 0.004 ms